In [ ]:
import urllib.request
import os.path
import numpy as np
import sys
sys.path.insert(0, './')
from utils import plot_mats
from pgpnn import ImageSplitter, PredictiveGatingPyramid

file_name = 'mnist_test_seq.npy'
url = 'http://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy'

if not os.path.isfile(file_name):
    print("could not find dataset: download it..")
    urllib.request.urlretrieve(url, file_name)
    print("download complete")

# Moving Mnist: 10.000 sequences of length 20 showing 2 digits moving in 64x64
moving_mnist = np.load(file_name) # shape: 20,10000,64,64
moving_mnist = np.rollaxis(moving_mnist, 1) # --> 10000,20,64,64

In [ ]:
model = PredictiveGatingPyramid(depth=2, modelname='test_pgp_300_runs_')
model.train(
    moving_mnist, 
    epochs=5,
    pre_epochs=25,
    learningRate=0.0001,
    save_results=False,
    load_first_stage=True,
    force_pretrain_first_stage=False)

[CONSTRUCT LAYER 1]
	could not preload weights for Layer 1
[CONSTRUCT LAYER 2]
	could not preload weights for Layer 2
Training: Epoch: 000/005 cost: 938.699735754


In [ ]:
# print(model.V1_np.shape)
# print(model.U1_np.shape)

# import matplotlib.pyplot as plt
# import pylab

# # function copied from the original code
# def dispims(M, height, width, border=0, bordercolor=0.0, layout=None, **kwargs):
#     from pylab import cm, ceil
#     numimages = M.shape[1]
#     if layout is None:
#         n0 = int(np.ceil(np.sqrt(numimages)))
#         n1 = int(np.ceil(np.sqrt(numimages)))
#     else:
#         n0, n1 = layout
#     im = bordercolor * np.ones(((height+border)*n0+border,(width+border)*n1+border),dtype='<f8')
#     for i in range(n0):
#         for j in range(n1):
#             if i*n1+j < M.shape[1]:
#                 im[i*(height+border)+border:(i+1)*(height+border)+border,
#                    j*(width+border)+border :(j+1)*(width+border)+border] = np.vstack((
#                             np.hstack((np.reshape(M[:,i*n1+j],(height, width)),
#                                    bordercolor*np.ones((height,border),dtype=float))),
#                             bordercolor*np.ones((border,width+border),dtype=float)
#                             ))
    
#     return im


# I = dispims(model.U1_np, 64, 64, 10)
# J = dispims(model.V1_np, 64, 64, 10)

# fig = plt.figure(figsize=(16,8))
# fig.add_subplot(121).imshow(I)
# fig.add_subplot(122).imshow(J)

# plt.show()